In [2]:
# list of letters A-K
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J','K']
# numbers 1 - 18
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

# create a list of all possible coordinates
KICKBOARD_HEIGHT = 370
WIDTH = 2440
HEIGHT = 3660

DEG_40 = 40
DEG_25 = 25

HEIGHT_40 = 3150
HEIGHT_25 = 3700
DEPTH_40 = 2350
DEPTH_25 = 1550

SPACING = 200

In [3]:
# create a pandas dataframe where each row is a hold, with columns for the hold's letter, number, and x-coordinates, y-xoordinates, and z-coordinates
import pandas as pd 
import math

def get_moonboard_hold_grid():
    # at 25 degrees, calculate x,y,z coordinates
    coordinates = []
    for i, letter in enumerate(letters):
        for j, number in enumerate(numbers):
            x = i*SPACING+SPACING
            y = KICKBOARD_HEIGHT + j*SPACING+SPACING
            # new y at 25 degrees is tha adjacent to the hypothenuse of the triangle formed by the board and the wall
            y_25 = KICKBOARD_HEIGHT + (j*SPACING+SPACING)*math.cos(math.radians(DEG_25))
            y_40 = KICKBOARD_HEIGHT + (j*SPACING+SPACING)*math.cos(math.radians(DEG_40))
            # z at 25 degrees is tha opposite to the hypothenuse of the triangle formed by the board and the wall
            z_25 = (j*SPACING+SPACING)*math.sin(math.radians(DEG_25))
            z_40 = (j*SPACING+SPACING)*math.sin(math.radians(DEG_25))
            coordinates.append([letter, number, x, y, y_25,z_25,y_40,z_40 ])


    # create a pandas dataframe where each row is a hold, with columns for the hold's letter, number, and x-coordinates, y-xoordinates, and z-coordinates
    holds = pd.DataFrame(coordinates, columns = ['letter', 'number', 'x', 'y', 'y_25', 'z_25', 'y_40', 'z_40'])
    # combine letter and number into single column
    holds['number'] = holds['letter'] + holds['number'].astype(str)
    return holds
    

In [5]:
hold_setup_2019_example = """
Original School Holds
1 - E2 - N
2 - D6 - NW
3 - B1 - NW
4 - G3 - N
5 - J16 - N
7 - K5 - N
Hold Set A
51 - J6 - N
52 - K4 - SE
53 - I2 - SE
54 - E12 - SW
56 - F3 - SW
57 - K15 - NE
59 - K17 - NE
"""

example = {
    '2019': {
      '1': {
          'position': 'H7',
          'compass': 'SE',
          'set': 'Original School Holds'
      },
      '2': {
          'position': 'J10',
          'compass': 'E',
          'set': 'Hold Set A'
      }
    }
}
# import hold setup from file ./hold_setup_2019.txt
# create a dictionary of hold setups
# each hold setup is a dictionary of holds
# each hold is a dictionary of position and compass and set

def import_hold_setups_from_file():
    """ imports hold setups as string from .txt file"""
    hold_setup_2016 = ""
    with open('./hold_setups/hold_setup_2016.txt', 'r') as file:
        hold_setup_2016 = file.read()
    hold_setup_2017 = ""
    with open('./hold_setups/hold_setup_2017.txt', 'r') as file:
        hold_setup_2017 = file.read()
    hold_setup_2019 = ""
    with open('./hold_setups/hold_setup_2019.txt', 'r') as file:
        hold_setup_2019 = file.read()
    
    return {'2019': hold_setup_2019, '2017': hold_setup_2017, '2016': hold_setup_2016}

def parse_hold_setup(hold_setup):
    """Converts text strings setup to dictionary with position and compass"""
    hold_setup = hold_setup.split('\n')
    hold_setup = [line for line in hold_setup if line != '']
    # if line is not empty, and does not conaint ' - ', then it is the current set
    current_set = ''
    for i, line in enumerate(hold_setup):
        if ' - ' not in line:
            current_set = line
            # remove line
            hold_setup[i] = ''
        else:
            hold_setup[i] = current_set + ' - ' + line
    hold_setup = [line for line in hold_setup if line != '']
    hold_setup = [line.split(' - ') for line in hold_setup]
    hold_setup = {line[2]: {'num': line[1],'set': line[0],'position': line[2], 'compass': line[3]} for line in hold_setup}
    return hold_setup

def import_hold_setups():
    """ imports hold setups as dictionary from .txt file"""
    hold_setups = import_hold_setups_from_file()
    hold_setups = {year: parse_hold_setup(hold_setup) for year, hold_setup in hold_setups.items()}
    print()
    return hold_setups


def get_hold_position(hold_setup, hold_number, year):
    """ returns the position of a hold given the hold setup and year"""
    return hold_setup[year][str(hold_number)]['position']

def get_hold_compass(hold_setup, hold_number, year):
    """ returns the compass of a hold given the hold setup and year"""
    return hold_setup[year][str(hold_number)]['compass']

hold_setups = import_hold_setups()
print(hold_setups['2019']['E4'])


{'num': 'w07', 'set': 'Wooden Holds', 'position': 'E4', 'compass': 'SW'}


In [6]:
# generate moonboards for 2019, 2017, 2016

def generate_moonboard(hold_setup, year):
    holds = get_moonboard_hold_grid()
    # for each hold in holds dataframe, check if it is in hold_setup. If so, add the columns for num, set and compass
    for i, row in holds.iterrows():
        if(str(row['number']) not in hold_setup[year]):
            hold_compass = ''
            hold_set = ''
        else:
            hold_compass = get_hold_compass(hold_setup, row['number'], year)
            hold_set = hold_setup[year][str(row['number'])]['set']
        holds.at[i,'compass'] = hold_compass
        holds.at[i,'set'] = hold_set
    return holds

moonboard_2019 = generate_moonboard(hold_setups, '2019')
moonboard_2019.head()

    # for each position, look to see if there is a hold there
    


,letter,number,x,y,y_25,z_25,y_40,z_40,compass,set
0,A,A0,200,570,551.261557,84.523652,523.208889,84.523652,,
1,A,A1,200,770,732.523115,169.047305,676.417777,169.047305,SW,Hold Set B
2,A,A2,200,970,913.784672,253.570957,829.626666,253.570957,NE,Hold Set A
3,A,A3,200,1170,1095.046230,338.094609,982.835554,338.094609,SW,Hold Set B
4,A,A4,200,1370,1276.307787,422.618262,1136.044443,422.618262,N,Hold Set A


In [9]:
# for each hold, create a lookup table for the distance betwene each hold
# the rows and columns for the df are the hold numbers

hold_dist_df_40 = pd.DataFrame(columns = moonboard_2019['number'], index = moonboard_2019['number'])

# calculate the distance between each hold in 3d space
for i, row in moonboard_2019.iterrows():
    for j, row2 in moonboard_2019.iterrows():
        # calculate distance between hold i and hold j
        dist = math.sqrt((row['x'] - row2['x'])**2 + (row['y_40'] - row2['y_40'])**2 + (row['z_40'] - row2['z_40'])**2)
        # rounded to nearest whole number
        hold_dist_df_40.at[row['number'], row2['number']] = round(dist)


hold_dist_df_40.head()

number,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,K9,K10,K11,K12,K13,K14,K15,K16,K17,K18
number,,,,,,,,,,,,,,,,,,,,,
A0,0,175,350,525,700,875,1050,1225,1400,1575,...,2546,2657,2776,2900,3029,3162,3300,3441,3584,3731
A1,175,0,175,350,525,700,875,1050,1225,1400,...,2441,2546,2657,2776,2900,3029,3162,3300,3441,3584
A2,350,175,0,175,350,525,700,875,1050,1225,...,2345,2441,2546,2657,2776,2900,3029,3162,3300,3441
A3,525,350,175,0,175,350,525,700,875,1050,...,2259,2345,2441,2546,2657,2776,2900,3029,3162,3300
A4,700,525,350,175,0,175,350,525,700,875,...,2183,2259,2345,2441,2546,2657,2776,2900,3029,3162


In [10]:
# export hold_dist_df_40 to csv
hold_dist_df_40.to_csv('./exports/hold_dist_df_40.csv')